Testing and hearing how the midi files are given to the network.

In [1]:
from pathlib import Path
import music21 as m21

NUM_PITCHES = 128
STEP_SIZE = 0.25
REST = "R"
HOLD = "H"


def read_midi_to_time_series(
    file: Path | str,
    rest=REST,
    hold=HOLD,
    step=STEP_SIZE,
    target_key=m21.pitch.Pitch("C"),
) -> list[int | tuple | str, float]:
    """Read midi file to a fixed-step time series representation

    Example: [52, 32, hold, hold, rest, 22, hold, ...] where all events are equidistant in time.

    Args:
        file: path to midi file
        rest: symbol for a rest
        hold: symbol for holding the previous note
        step: Step size for time series (sampling step). Fraction of quarter note. Durations smaller than step are quantized to step. Defaults to 0.25.
        target_key: Key to transpose to. Defaults to C major / A minor.

    Returns:
        list of tokens (midi note numbers, rests, or holds)
    """
    song = m21.converter.parse(file)
    if target_key is not None:
        song = transpose_song(song, target_key=target_key)
    instruments = m21.instrument.partitionByInstrument(song).parts
    instrument = instruments[0]  # Use first instrument.
    notes = []
    for event in instrument.recurse():
        if isinstance(event, m21.note.Note):
            note = event.pitch.midi
        elif isinstance(event, m21.note.Rest):
            note = rest
        elif isinstance(event, m21.chord.Chord):
            # note = tuple(n.pitch.midi for n in event.notes) # Save all notes as tuple.
            note = event.notes[0].pitch.midi  # Save first note in chord.
        else:
            continue
        duration = event.duration.quarterLength
        num_steps = max(1, int(duration / step))
        notes.append(note)
        notes.extend([hold] * (num_steps - 1))
    return notes


def transpose_song(song: m21.stream, target_key=m21.pitch.Pitch("C")) -> m21.stream:
    """Transpose a music21 stream to a target key

    Args:
        song: music21 stream
        target_key: Target key. Defaults to C major / A minor.

    Returns:
        transposed song (music21 stream)
    """
    key = song.analyze("key")
    if key.mode == "major":
        return song.transpose(m21.interval.Interval(key.tonic, target_key))
    elif key.mode == "minor":
        return song.transpose(
            m21.interval.Interval(key.tonic, target_key.transpose(-3))
        )


def time_series_to_midi(
    sequence: list[str],
    step_duration: float = STEP_SIZE,
    filename: str | Path = None,
    hold_token=HOLD,
    rest_token=REST,
):
    """Convert a time series melody to midi

    Args:
        sequence: list of strings. A melody as notes or rests or hold tokens at fixed time steps.
        filename: Path to save midi file. Defaults to None.

    Returns:
        music21 stream
    """
    stream = m21.stream.Stream()

    step = 1
    for e in sequence:
        if e == hold_token:
            step += 1
        else:
            length = step_duration * step
            if e == rest_token:
                note = m21.note.Rest(quarterLength=length)
            else:
                note = m21.note.Note(pitch=int(e), quarterLength=length)
            stream.append(note)
            step = 1

    if filename is not None:
        stream.write(fmt="midi", fp=filename)
    return stream

In [2]:
path_to_raw = Path("/Users/savv/datasets/maestro-v3.0.0")
files_raw = list(path_to_raw.glob("**/*.midi"))

In [4]:
file = files_raw[1]

print("Original")
m21.converter.parse(file).show("midi")

print(
    f"Processed: if chord then take first note, quantize to {100*STEP_SIZE:.0f}% of a quarter note."
)
ts = read_midi_to_time_series(file)
time_series_to_midi(ts[:1000]).show("midi")

Original


Processed: if chord then take first note, quantize to 25% of a quarter note.
